# Yeast cells detection sample notebook

In [2]:
import sys
sys.path.append('../')
#sys.path.append('%HOMEPATH%/detectron2') #insert local path to detecron2
import numpy as np

import yeastcells.clustering as clustering
import yeastcells.features as features
import yeastcells.data as data
import yeastcells.model as model
import yeastcells.output as output
import yeastcells.remove_false_pos as remove_fp
import yeastcells.visualize as visualize

### Load data

In [11]:
path = '../images/samples/Movie_1'
filenames = data.load_data(path)
print(filenames)

['../images/samples/Movie_1//12.4.2017_sfp1_ph-tdgfp_hta2_mcherry_1pc_glu_to_1pc_glu_rap_xy01.nd2.tif']


In [12]:
fn = filenames[0]
image = data.read_image(fn)
np.shape(image) #(frames, length, width, channels)

(217, 512, 512, 3)

### Load and run model on data for segmentation

In [ ]:
model_path = '../model'
predictor = model.load_model(model_path, seg_thresh=0.94, device='cuda:1')

In [18]:
output = [
  {'instances': predictor(frame)['instances'].to('cpu')}
  for frame in image
]

### Run clustering algorithm for tracking

In [ ]:
labels, coordinates = clustering.cluster_cells(output, dmax=5, min_samples=3, eps=0.6, progress=False)

### Reformat prediction outputs into array

In [ ]:
pred_s, pred_t = data.get_pred(output, labels, coordinates)

### Extract features and visualize tracking and segmentation

In [ ]:
n_tracks = features.get_tracks(labels)
print(n_tracks)

In [ ]:
contours = features.extract_contours(output) 

In [ ]:
polygons =  features.polygons_per_cluster(labels, countours, output, include_noise=False)

In [ ]:
#plot paths
ax = visualize.plot_paths(
  labels,
  coordinates[:, [2,1,0]],
  style={'linewidth': 1},
  xlim=(0, 512),
  ylim=(0, 512),
  title='',
)

In [ ]:
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=None, framenum=True, labelnum=False)
movie = visualize.show_animation(scene, title='', delay=500) 

In [ ]:
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=None, framenum=True, labelnum=True)
cell_scene, label = visualize.select_cell(coordinates, labels, w=40, l=0) # choose image window (w) and label (l)
movie = visualize.show_animation(cell_scene, title='', delay=500)

In [ ]:
ax = visualize.plot_area_profile(polygons, label=0, ax=None, title=None)

In [ ]:
poly_area, growth_rate = features.get_area_and_growth_rate(polygons, labels, pred_s)